In [1]:
import numpy as np
import pandas as pd
from pathlib import Path
from scoring import local_scorer
from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')

In [2]:
oof_cb = np.load('oof_cb.npy')
oof_lgb_init = np.load('oof_lgb-final.npy')
oof_lgb = oof_lgb_init.mean(axis=1)
oof_cb.shape, oof_lgb.shape

((21498,), (21498,))

In [3]:
INPUT_PATH = Path('../input')

funnel = pd.read_csv(INPUT_PATH / 'funnel.csv')
Y_test = funnel[['client_id', 'sale_flg']].set_index('client_id')
test_funnel =  funnel.set_index('client_id')
Y_test.shape

(21498, 1)

In [4]:
# prediction_threshold = 0.16
w_range = np.linspace(0, 1, 11)
t_range = np.linspace(0, 1, 101)[10:40]
results = []
for w in w_range:
    for t in tqdm(t_range):
        Y_predicted = (w * oof_cb + (1 - w) * oof_lgb > t).astype(int)
        public_score, private_score = local_scorer.get_score(test_funnel, Y_predicted, Y_test)
        anic = public_score / 3 + private_score * 2 / 3
        results.append((w, t, anic))
    # validation_precision = precision_score(Y_test['sale_flg'], Y_predicted)
    # validation_recall = recall_score(Y_test['sale_flg'], Y_predicted)
    # validation_f1 = f1_score(Y_test['sale_flg'], Y_predicted)
    # print(f'Public ANIC {public_score} Private ANIC {private_score}')
    # print(f'ANIC {1/3*public_score+ 2/3 * private_score}')
    # print(f'Precision {validation_precision} | Recall {validation_recall} | F1 {validation_f1}')

100%|██████████| 30/30 [00:45<00:00,  1.52s/it]


In [5]:
sorted(results, key=lambda x: x[2], reverse=True)[0]

(0.30000000000000004, 0.15, 5703.703795701927)

In [7]:
# prediction_threshold = 0.16
w_range = np.linspace(0, 1, 101)
t_range = [0.15]
results = []
for w in tqdm(w_range):
    for t in t_range:
        Y_predicted = (w * oof_cb + (1 - w) * oof_lgb > t).astype(int)
        public_score, private_score = local_scorer.get_score(test_funnel, Y_predicted, Y_test)
        anic = public_score / 3 + private_score * 2 / 3
        results.append((w, t, anic))
    # validation_precision = precision_score(Y_test['sale_flg'], Y_predicted)
    # validation_recall = recall_score(Y_test['sale_flg'], Y_predicted)
    # validation_f1 = f1_score(Y_test['sale_flg'], Y_predicted)
    # print(f'Public ANIC {public_score} Private ANIC {private_score}')
    # print(f'ANIC {1/3*public_score+ 2/3 * private_score}')
    # print(f'Precision {validation_precision} | Recall {validation_recall} | F1 {validation_f1}')

100%|██████████| 101/101 [02:34<00:00,  1.53s/it]


In [8]:
sorted(results, key=lambda x: x[2], reverse=True)[0]

(0.28, 0.15, 5705.192306261049)